# Teste Unitário - Aquisição

**Objetivo**: Gerar as bases amostrais do pacote de aquisição

---

## Cabeçalho 

### Imports 

In [1]:
import gc
import os
import pyunpack
import rarfile
import shutil
import sys
import zipfile
import numpy as np
import pandas as pd
import geopandas as gpd

from io import BytesIO
from pathlib import Path
from tqdm import tqdm

### Configurações 

In [2]:
%config Completer.use_jedi = False

### Caminhos 

In [3]:
try:
    PASTA_NOTEBOOK / ""
except:
    PASTA_NOTEBOOK = Path(os.getcwd())
    
PASTA_PROJETO = PASTA_NOTEBOOK.parent
PASTA_DADOS = PASTA_PROJETO / "dados"
PASTA_EXTERNO = PASTA_DADOS / "externo"
PASTA_TESTE = PASTA_PROJETO / "src/tests"
PASTA_TESTE_CODIGO = PASTA_TESTE / "codigos"
PASTA_TESTE_DADOS = PASTA_TESTE / "dados"

PASTA_TESTE_CODIGO.mkdir(exist_ok=True)
PASTA_TESTE_DADOS.mkdir(exist_ok=True)

os.chdir(PASTA_PROJETO)

---

## Códigos 

In [ ]:
docentes = pd.read_pickle(PASTA_TESTE_CODIGO / "docentes.pkl").values
escolas = pd.read_pickle(PASTA_TESTE_CODIGO / "escolas.pkl").values
gestores = pd.read_pickle(PASTA_TESTE_CODIGO / "gestores.pkl").values
matriculas = pd.read_pickle(PASTA_TESTE_CODIGO / "matriculas.pkl").values
turmas = pd.read_pickle(PASTA_TESTE_CODIGO / "turmas.pkl").values

---

## Censo Escolar 

In [ ]:
caminho_entrada = PASTA_EXTERNO / "censo_escolar"
caminho_saida = PASTA_TESTE_DADOS / "externo/censo_escolar"
caminho_saida.mkdir(exist_ok=True, parents=True)
tabelas = {
    "docentes_co": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_nordeste": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_norte": [docentes, ("ID_DOCETE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_sudeste": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "docentes_sul": [docentes, ("ID_DOCENTE", "FK_COD_DOCENTE", "CO_PESSOA_FISICA")], 
    "escolas": [escolas, ("PK_COD_ENTIDADE", "CO_ENTIDADE")], 
    "gestor": [gestores, ("ID_GESTOR",)], 
    "matricula_co": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_nordeste": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_norte": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_sudeste": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "matricula_sul": [matriculas, ("PK_COD_MATRICULA", "ID_MATRICULA")], 
    "turmas": [turmas, ("ID_TURMA", "PK_COD_TURMA")], 
}

# para cada arquivo do censo demográfico
for censo in tqdm(os.listdir(caminho_entrada)):
    # abre o arquivo zip com o conteúdo do censo
    with zipfile.ZipFile(caminho_entrada / f"{censo}") as zin:
        # lista os arquivos que precisam de uma versão amostral
        arqs = [
            f 
            for tabela in tabelas
            for f in zin.namelist() 
            if f"{tabela}." in f.lower()
        ]
        
        if len(arqs) == 0:
            continue
        
        # cria uma cópia do arquivo zip na pasta de destino
        with zipfile.ZipFile(
            caminho_saida / f"{censo}", 
            mode="w", 
            compression=zipfile.ZIP_DEFLATED, 
            compresslevel=9
        ) as zout:
            # para cada item no arquivo de entrada
            # for item in zin.infolist():
            #     # verifica se o item faz parte de um dos arquivos amostrais
            #     if item.filename not in arqs:
            #         # se não, o copia para o novo zip
            #         zout.writestr(item, zin.read(item.filename))
            
            # para os demais itens
            for arq in arqs:
                tabela = arq.lower().split("/")[-1].split(".")[0]
                arq_zip = arq.split("/")[-1].split(".")[0]
                
                # carrega o arquivo CSV
                if ".csv" in arq.lower():
                    df = pd.read_csv(zin.open(arq), encoding="latin-1", sep="|")
                elif ".zip" in arq.lower():
                    with zipfile.ZipFile(zin.open(arq)) as z2:
                        para_abrir = z2.namelist()[0]
                        df = pd.read_csv(z2.open(para_abrir), encoding="latin-1", sep="|")
                elif ".rar" in arq.lower():
                    zin.extract(arq, path=caminho_entrada)
                    pyunpack.Archive(caminho_entrada / f"{arq}").extractall(caminho_entrada)
                    csv = [
                        f
                        for f in os.listdir(caminho_entrada)
                        if f"{tabela}." in f.lower()
                    ][0]
                    df = pd.read_csv(caminho_entrada / f"{csv}", encoding="latin-1", sep="|")
                    shutil.rmtree(caminho_entrada / f"{arq.split('/')[0]}")
                    os.remove(caminho_entrada / f"{csv}")

                # aplica os filtros sobre o arquivo
                col = [c for c in tabelas[tabela][1] if c in df][0]
                df = df.loc[lambda f: f[col].isin(tabelas[tabela][0])]
                
                # exporta o resultado
                # df.to_parquet(PASTA_TESTE_AQUIS / f"intermediario/{tabela}.parquet", index=False)
                if ".csv" in arq.lower():
                    txt = df.to_csv(encoding="latin-1", sep="|", index=False)
                    zout.writestr(arq, txt)
                elif ".zip" in arq.lower():
                    with zipfile.ZipFile(
                        caminho_saida / f"{arq_zip}.zip", 
                        mode="w", 
                        compression=zipfile.ZIP_DEFLATED, 
                        compresslevel=9
                    ) as zinner:
                        txt = df.to_csv(encoding="latin-1", sep="|", index=False)
                        zinner.writestr(f"{arq_zip}.csv", txt)
                    zout.write(f"{caminho_saida / arq_zip}.zip", arq)
                    os.remove(f"{caminho_saida / arq_zip}.zip")
                elif ".rar" in arq.lower():
                    os.chdir(caminho_saida)
                    df.to_csv(f"{arq_zip}.csv", encoding="latin-1", sep="|", index=False)
                    if "win" in sys.platform:
                        os.system(f"{PASTA_PROJETO / 'suporte/WinRAR/rar'} a {arq_zip}.rar {arq_zip}.csv")
                    else:
                        os.system(f"rar a {arq_zip}.rar {arq_zip}.csv")
                    zout.write(f"{caminho_saida / arq_zip}.rar", arq)
                    os.remove(f"{caminho_saida / arq_zip}.csv")
                    os.remove(f"{caminho_saida / arq_zip}.rar")
                    os.chdir(PASTA_PROJETO)

---

## IDEB 

In [ ]:
caminho_entrada = PASTA_EXTERNO / "ideb"
caminho_saida = PASTA_TESTE_DADOS / "externo/ideb"
caminho_saida.mkdir(exist_ok=True, parents=True)

for f in tqdm(os.listdir(caminho_entrada)):
    nome = os.path.splitext(f)[0]
    with zipfile.ZipFile(caminho_entrada / f) as z:
        data = pd.read_excel(z.open(f"{nome}/{nome}.xlsx"), header=None)
        sample = (
            data.head(10)
            .append(data[data[3].astype("str").isin(escolas.astype("str"))])
            .append(data.tail(3))
        )
        
    with zipfile.ZipFile(
        caminho_saida / f, 
        mode="w", 
        compression=zipfile.ZIP_DEFLATED, 
        compresslevel=9
    ) as zout:
        sample.to_excel(caminho_saida / f"{nome}.xlsx", header=None, index=False)
        zout.write(caminho_saida / f"{nome}.xlsx", f"{nome}/{nome}.xlsx")
        os.remove(caminho_saida / f"{nome}.xlsx")

## Malha Município 

In [28]:
df = gpd.read_file("zip://dados/externo/ibge/BR_Pais_2021.zip")

In [29]:
df

,NM_PAIS,AREA_KM2,geometry
0,Brasil,8510345.54,"MULTIPOLYGON (((-49.71294 -29.32558, -49.71257..."


In [10]:
p = df.centroid.values[0]

C:\Users\pedro.forli\AppData\Local\Temp/ipykernel_9656/2969920936.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  p = df.centroid.values[0]


In [25]:
from pyproj import Geod
from shapely import wkt

# specify a named ellipsoid
geod = Geod(ellps="WGS84")

In [23]:
poly = df["geometry"].values[0]

In [27]:
abs(geod.geometry_area_perimeter(poly)[0]) / 1000000

7067.126907547882

In [22]:
df

,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry
0,1100015,Alta Floresta D'Oeste,RO,7067.127,"POLYGON ((-62.19465 -11.82746, -62.19332 -11.8..."
1,1100023,Ariquemes,RO,4426.571,"POLYGON ((-62.53648 -9.73222, -62.52765 -9.736..."
2,1100031,Cabixi,RO,1314.352,"POLYGON ((-60.37119 -13.36655, -60.37134 -13.3..."
3,1100049,Cacoal,RO,3793.000,"POLYGON ((-61.00080 -11.29737, -61.00103 -11.3..."
4,1100056,Cerejeiras,RO,2783.300,"POLYGON ((-61.49976 -13.00525, -61.49809 -13.0..."
...,...,...,...,...,...
5567,5222005,Vianópolis,GO,954.115,"POLYGON ((-48.42801 -16.62789, -48.42803 -16.6..."
5568,5222054,Vicentinópolis,GO,733.794,"POLYGON ((-49.84740 -17.57796, -49.84719 -17.5..."
5569,5222203,Vila Boa,GO,1052.593,"POLYGON ((-47.10998 -14.67195, -47.10994 -14.6..."
5570,5222302,Vila Propício,GO,2181.593,"POLYGON ((-48.75572 -14.86393, -48.75502 -14.8..."


In [21]:
df.area *

C:\Users\pedro.forli\AppData\Local\Temp/ipykernel_9656/2704296042.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df.area


0       0.587657
1       0.364965
2       0.109724
3       0.314085
4       0.232089
          ...   
5567    0.080884
5568    0.062507
5569    0.088448
5570    0.183550
5571    0.485863
Length: 5572, dtype: float64

In [16]:
(
    df.rename(columns={
        "CD_UF": "CO_UF",
        "NM_UF": "NO_UF",
        "NM_REGIAO": "NO_REGIAO",
        "SIGLA": "UF"
    })
    .assign(
        CO_UF=lambda f: f["CO_UF"].astype(int),
        AREA_KM2=lambda f: f.area,
        LATITUDE=lambda f: f["geometry"].apply(lambda x: x.centroid.coords[0][1]),
        LONGITUDE=lambda f: f["geometry"].apply(lambda x: x.centroid.coords[0][0])
    )
)

,CO_MUNICIPIO,NM_MUN,UF,AREA_KM2,geometry,LATITUDE,LONGITUDE
0,1100015,Alta Floresta D'Oeste,RO,7067.127,"POLYGON ((-62.19465 -11.82746, -62.19332 -11.8...",-12.470132,-62.274661
1,1100023,Ariquemes,RO,4426.571,"POLYGON ((-62.53648 -9.73222, -62.52765 -9.736...",-9.951890,-62.957255
2,1100031,Cabixi,RO,1314.352,"POLYGON ((-60.37119 -13.36655, -60.37134 -13.3...",-13.474879,-60.639807
3,1100049,Cacoal,RO,3793.000,"POLYGON ((-61.00080 -11.29737, -61.00103 -11.3...",-11.301287,-61.324729
4,1100056,Cerejeiras,RO,2783.300,"POLYGON ((-61.49976 -13.00525, -61.49809 -13.0...",-13.203562,-61.260950
...,...,...,...,...,...,...,...
5567,5222005,Vianópolis,GO,954.115,"POLYGON ((-48.42801 -16.62789, -48.42803 -16.6...",-16.812810,-48.440772
5568,5222054,Vicentinópolis,GO,733.794,"POLYGON ((-49.84740 -17.57796, -49.84719 -17.5...",-17.723637,-49.872692
5569,5222203,Vila Boa,GO,1052.593,"POLYGON ((-47.10998 -14.67195, -47.10994 -14.6...",-14.993116,-47.062373
5570,5222302,Vila Propício,GO,2181.593,"POLYGON ((-48.75572 -14.86393, -48.75502 -14.8...",-15.269677,-48.813538


---